In [1]:
# ## imports
import os
import tensorflow as tf

from models.VAE_a import VariationalAutoencoder
from utils.loaders_a import load_mnist
from utils.gpu_utils import gpu_optim
from utils.custom_utils import mk_run_folders
from utils.custom_utils import timer, benchmark, copy_weights, try_func


In [2]:
# project dirs

gpu_flag = True
build_flag = False  # build / load
train_flag = True

SECTION = 'vae'
RUN_ID = '0002'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

mk_run_folders(RUN_FOLDER, 'viz', 'images', 'weights')

if gpu_flag:
    gpu_optim()  # gpu start


run dirs OK
1 Physical GPUs, 1 Logical GPUs


In [3]:
# params

EPOCHS = 1
BATCH_SIZE = 32
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0
LEARNING_RATE = 0.0005


# ## data load
(x_train, y_train), (x_test, y_test) = load_mnist()


In [4]:
# ## architecture

vae = VariationalAutoencoder(
    input_dim=(28, 28, 1)
    , encoder_conv_filters=[32, 64, 64, 64]
    , encoder_conv_kernel_size=[3, 3, 3, 3]
    , encoder_conv_strides=[1, 2, 2, 1]
    , decoder_conv_t_filters=[64, 64, 32, 1]
    , decoder_conv_t_kernel_size=[3, 3, 3, 3]
    , decoder_conv_t_strides=[1, 2, 2, 1]
    , z_dim=2
    , r_loss_factor=1000
)

vae.compile(LEARNING_RATE)

if build_flag:
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights_'))
    print("wights loaded: ", os.path.join(RUN_FOLDER, 'weights/weights_'))

vae.encoder.summary()
vae.decoder.summary()


wights loaded:  run/vae/0002_digits\weights/weights_
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 28, 28, 32)   0           encoder_conv_0[0][0]             
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 14, 14, 64)   18496       leaky_re_lu[0][0]                
_______________________________________

In [5]:
# ## training

vae.train = timer(vae.train)  # timer decorator func

if train_flag:
    vae.train(
        x_train
        , batch_size=BATCH_SIZE
        , epochs=EPOCHS
        , run_folder=RUN_FOLDER
        , print_every_n_batches=PRINT_EVERY_N_BATCHES
        , initial_epoch=INITIAL_EPOCH
    )


   1/1875 [..............................] - ETA: 1s - loss: 40.7345 - reconstruction_loss: 35.2928 - kl_loss: 5.4418

   9/1875 [..............................] - ETA: 11s - loss: 41.7488 - reconstruction_loss: 36.0429 - kl_loss: 5.7059

  17/1875 [..............................] - ETA: 11s - loss: 42.0188 - reconstruction_loss: 36.3271 - kl_loss: 5.6918

  25/1875 [..............................] - ETA: 11s - loss: 42.0352 - reconstruction_loss: 36.3434 - kl_loss: 5.6918

  34/1875 [..............................] - ETA: 11s - loss: 42.1419 - reconstruction_loss: 36.4417 - kl_loss: 5.7003

...

1856/1875 [============================>.] - ETA: 0s - loss: 43.1051 - reconstruction_loss: 37.5001 - kl_loss: 5.6050

1865/1875 [============================>.] - ETA: 0s - loss: 43.0933 - reconstruction_loss: 37.4875 - kl_loss: 5.6058

1875/1875 [==============================] - ETA: 0s - loss: 43.0957 - reconstruction_loss: 37.4899 - kl_loss: 5.6058


Epoch 00001: saving model to run/vae/0002_digits\weights\weights
1875/1875 [==============================] - 11s 6ms/step - loss: 43.0958 - reconstruction_loss: 37.4900 - kl_loss: 5.6057


Time elapsed: min 0, sec 13


In [6]:
# copy weights

copy_weights(RUN_FOLDER, add='_')

copy OK: run/vae/0002_digits\weights/weights_.data-00000-of-00001


In [7]:
tf.keras.backend.clear_session()
